In [ ]:
!pip install uv && uv pip install sovai['full'] --system > output.log 2>&1

## SEC 10-K filings

You can run the following commands to retrieve data (`df`) using `sov.data`:

To fetch the **latest data** for a specific query:

```python
df = sov.data("query")
```

To fetch the **full historical data** for a specific query:

```python
df = sov.data("query", full_history=True)
```

To fetch the **full data** multiple **tickers** or identifiers like **cusip** and **openfigi**:

```python
df = sov.data("query", tickers=["9033434", "IB94343", "43432", "AAPL"])
```

To filter **any dataframe** just write some queries:

```python
df.filter(["cash_short_term > 10m","start with ticker A","negative profits" ])
```


In [5]:
import sovai as sov
import pandas as pd

sov.token_auth(token="visit https://sov.ai/profile for your token")

data = sov.data("sec/10k", tickers=["AAPL"], limit=1).reset_index()

In [6]:
data

,ticker,date,cik,company,accession_number,section,order,category,tag,row_id,full_text
0,AAPL,2011-10-26,320193,APPLE INC,0001193125-11-282113,BUSINESS,0,NarrativeText,p,e19f7471d8fb867c5bc0026bc798ad53,Apple Inc. and its wholly-owned subsidiaries (...
1,AAPL,2011-10-26,320193,APPLE INC,0001193125-11-282113,BUSINESS,1,NarrativeText,p,1ce3d4c53620d2d099ab59da4ebc3b9e,The Company is committed to bringing the best ...
2,AAPL,2011-10-26,320193,APPLE INC,0001193125-11-282113,BUSINESS,2,NarrativeText,p,a8eda7d4be126f5088578bb5b355dc0d,The Company believes a high-quality buying exp...
3,AAPL,2011-10-26,320193,APPLE INC,0001193125-11-282113,BUSINESS,3,NarrativeText,p,834e8b74a0fbb0cdd5fd390f1979bfac,The Company’s retail stores are typically loca...
4,AAPL,2011-10-26,320193,APPLE INC,0001193125-11-282113,BUSINESS,4,NarrativeText,p,1e2460c44fe3255902521a729dabc0ae,"Throughout its history, the Company has been c..."
...,...,...,...,...,...,...,...,...,...,...,...
7700,AAPL,2023-11-03,320193,Apple Inc.,0000320193-23-000106,MANAGEMENT,54,NarrativeText,span,0cedd2f8c148b1fc82f4dc0f5c9f35b8,The Company is subject to income taxes in the ...
7701,AAPL,2023-11-03,320193,Apple Inc.,0000320193-23-000106,MANAGEMENT,55,NarrativeText,span,b771b34af4ca51d17d3fbf42777c02c0,The Company is subject to various legal procee...
7702,AAPL,2023-11-03,320193,Apple Inc.,0000320193-23-000106,COMPENSATION,0,NarrativeText,span,f8825199078db1af16a75e19d38c8eff,The information required by this Item will be ...
7703,AAPL,2023-11-03,320193,Apple Inc.,0000320193-23-000106,PRINCIPAL_STOCKHOLDERS,0,NarrativeText,span,f8825199078db1af16a75e19d38c8eff,The information required by this Item will be ...


You don't have to downlaod the entire dataset, instead look at the companies that filed that week, and simply do an analysis of them. Future edits would include sentiment analysis.

In [7]:
import pandas as pd

# Define mandatory sections
mandatory_sections = [
    "FINANCIAL_STATEMENTS",
    "RISK_FACTORS",
    "MANAGEMENT_DISCUSSION",
    "MANAGEMENT",
    "COMPENSATION",
    "EXHIBITS",
    "BUSINESS",
    "CONTROLS_AND_PROCEDURES",
    "PRINCIPAL_STOCKHOLDERS",
    "ACCOUNTING_FEES"
]

# 1. Filter for mandatory sections using isin (vectorized operation)
mandatory_data = data[data['section'].isin(mandatory_sections)].copy()

# 2. Extract year from the 'date' column
mandatory_data['year'] = mandatory_data['date'].dt.year

# 3. Calculate word count using vectorized string operations
mandatory_data['word_count'] = mandatory_data['full_text'].str.count(r'\w+')

# 4. Group by 'ticker', 'year', and 'section', then sum the 'word_count'
word_count_agg = mandatory_data.groupby(['ticker', 'year', 'section'], as_index=False)['word_count'].sum()

# 5. Pivot the DataFrame to create a multi-index with 'ticker' and 'year'
word_count_pivot = word_count_agg.pivot_table(
    index=['ticker', 'year'],
    columns='section',
    values='word_count',
    fill_value=0  # Fill missing values with 0
)

# Optional: Flatten the column index if needed (not necessary in multi-index)
# word_count_pivot.columns = word_count_pivot.columns.get_level_values(0)

# 6. Reset index to ensure 'ticker' and 'year' are part of the index
word_count_pivot = word_count_pivot.reset_index().set_index(['ticker', 'year'])


In [8]:
word_count_pivot

section      ACCOUNTING_FEES  BUSINESS  COMPENSATION  CONTROLS_AND_PROCEDURES  \
ticker year                                                                     
AAPL   2011           29.000  4839.000     13454.000                  646.000   
       2012           76.000  4763.000     13181.000                  644.000   
       2013            0.000   563.000         0.000                  652.000   
       2014            0.000   582.000         0.000                  653.000   
       2015            0.000   550.000         0.000                  653.000   
       2016           18.000   537.000        18.000                  635.000   
       2017           18.000   552.000        26.000                  635.000   
       2018           18.000   545.000        26.000                  635.000   
       2019           18.000   345.000        26.000                  635.000   
       2020            0.000   297.000        77.000                  642.000   
       2021            0.000   184.000        34.000                  642.000   
       2022            0.000   920.000        20.000                  642.000   
       2023            0.000   442.000        20.000                  636.000   

section      EXHIBITS  FINANCIAL_STATEMENTS  MANAGEMENT  \
ticker year                                               
AAPL   2011 15685.000                41.000    8740.000   
       2012  5485.000                57.000    9074.000   
       2013     0.000             13676.000    8916.000   
       2014     0.000             14159.000    9194.000   
       2015     0.000             13244.000    8300.000   
       2016     0.000             11799.000    8005.000   
       2017     0.000             11927.000    8091.000   
       2018     0.000             10412.000    7470.000   
       2019     0.000             10338.000    4183.000   
       2020     0.000             11290.000    4381.000   
       2021     0.000              9289.000    2553.000   
       2022     0.000              8192.000    2348.000   
       2023     0.000              6678.000    2123.000   

section      MANAGEMENT_DISCUSSION  PRINCIPAL_STOCKHOLDERS  RISK_FACTORS  
ticker year                                                               
AAPL   2011               8740.000                   0.000      7529.000  
       2012               9074.000                  51.000      7177.000  
       2013               8916.000                  51.000      7532.000  
       2014               9194.000                  51.000      7866.000  
       2015               8300.000                  51.000      7931.000  
       2016               8005.000                  18.000      7769.000  
       2017               8091.000                  18.000      7873.000  
       2018               7470.000                  18.000      8172.000  
       2019               4183.000                  18.000      8114.000  
       2020               4381.000                   0.000      9015.000  
       2021               2553.000                  34.000      9835.000  
       2022               2348.000                  20.000     10405.000  
       2023               2123.000                  20.000      9951.000

In [9]:
sov.data("sec/10k", tickers=["META"], limit=1)

cik               company      accession_number  \
ticker date                                                              
META   2013-02-01  1326801          Facebook Inc  0001326801-13-000003   
       2013-02-01  1326801          Facebook Inc  0001326801-13-000003   
       2013-02-01  1326801          Facebook Inc  0001326801-13-000003   
       2013-02-01  1326801          Facebook Inc  0001326801-13-000003   
       2013-02-01  1326801          Facebook Inc  0001326801-13-000003   
...                    ...                   ...                   ...   
       2024-02-02  1326801  Meta Platforms, Inc.  0001326801-24-000012   
       2024-02-02  1326801  Meta Platforms, Inc.  0001326801-24-000012   
       2024-02-02  1326801  Meta Platforms, Inc.  0001326801-24-000012   
       2024-02-02  1326801  Meta Platforms, Inc.  0001326801-24-000012   
       2024-02-02  1326801  Meta Platforms, Inc.  0001326801-24-000012   

                             section  order       category   tag  \
ticker date                                                        
META   2013-02-01           BUSINESS      0  NarrativeText  font   
       2013-02-01         PROPERTIES      0  NarrativeText  font   
       2013-02-01  LEGAL_PROCEEDINGS      0  NarrativeText  font   
       2013-02-01  LEGAL_PROCEEDINGS      1  NarrativeText  font   
       2013-02-01  LEGAL_PROCEEDINGS      2  NarrativeText  font   
...                              ...    ...            ...   ...   
       2024-02-02           EXHIBITS      3  NarrativeText    td   
       2024-02-02           EXHIBITS      4  NarrativeText    td   
       2024-02-02           EXHIBITS      5  NarrativeText    td   
       2024-02-02           EXHIBITS      6  NarrativeText  span   
       2024-02-02           EXHIBITS      7  NarrativeText  span   

                                             row_id  \
ticker date                                           
META   2013-02-01  a6145cfc83491272af346a0eae2c3336   
       2013-02-01  e0e664ee42b78d089f4ac1248e533e86   
       2013-02-01  8ae3ba693a1fbd26c64a636d5c0d7aca   
       2013-02-01  0dc121ed1c32cf81c4195c359707ec12   
       2013-02-01  322930695bbf6c305a377de61245609b   
...                                             ...   
       2024-02-02  efc44904ae01a497b5549538d5938573   
       2024-02-02  83e1d06a719c9d49143c8e856969b30c   
       2024-02-02  7a2dae8e4a37cb40603f71c65bb2a9ed   
       2024-02-02  ed1d31dff6a985dfad8bc386b6c565fd   
       2024-02-02  cce6c86af38a0cefa0517717cf2ea244   

                                                           full_text  
ticker date                                                           
META   2013-02-01  We allocate the fair value of purchase conside...  
       2013-02-01  As of December 31, 2012, we leased office and ...  
       2013-02-01  Paul D. Ceglia filed suit against us and Mark ...  
       2013-02-01  Beginning on May 22, 2012, multiple putative c...  
       2013-02-01  We are also party to various legal proceedings...  
...                                                              ...  
       2024-02-02  Certification of Susan Li, Chief Financial Off...  
       2024-02-02  Certification of Mark Zuckerberg, Chief Execut...  
       2024-02-02  Certification of Susan Li, Chief Financial Off...  
       2024-02-02  + Indicates a management contract or compensat...  
       2024-02-02  # This certification is deemed not filed for p...  

[10550 rows x 9 columns]